In [31]:
from PIL import Image
from torchvision import datasets, transforms
from torch.utils.data import Dataset,DataLoader,TensorDataset
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch

In [32]:
class OmniglotDataset(Dataset):
    def __init__(self, transform, samples):
        self.transform = transform
        self.samples = samples
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        img_path, alphabet, label = self.samples[index]
        img = Image.open(img_path).convert('L')
        img = self.transform(img)
        
        return img,alphabet,label

In [33]:
from torchvision.transforms import transforms, InterpolationMode
samples = "/Users/abhishekwaghchaure/Desktop/My_practice_noteooks/RNN/datasets/omniglot_train"

dataset_train = OmniglotDataset(
    transform = transforms.Compose([
        transforms.Resize((64, 64), interpolation=InterpolationMode.BILINEAR),  # Correct size as a tuple and interpolation mode
        transforms.ToTensor()
    ]),
    samples = samples
)


In [36]:
train_loader = DataLoader(dataset_train, shuffle = True, batch_size= 3)

In [35]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.image_layer = nn.Sequential(
            nn.Conv2d(1,16, kernel_size=(3,3), padding = 1),
            nn.MaxPool2d(kernel_size=(2,2)),
            nn.ELU(),
            
            nn.Flatten(),
            nn.Linear(16 * 32*32, 128)
        )
        
        self.alphabet_layer = nn.Sequential(
            nn.Linear(30,8),
            nn.ELU()
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 + 8, 964)
        )
        
    def forward(self, x_image,x_alphabet):
        x_image = self.image_layer(x_image)
        x_alphabet = self.alphabet_layer(x_alphabet)
        x = torch.cat((x_image, x_alphabet), dim = 1)
        
        return self.classifier(x)

In [41]:
from tqdm import tqdm
from torchmetrics import Accuracy

In [42]:
device = torch.device = "mps"

In [43]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Net.parameters(), lr = 0.001)
acc = Accuracy(task='multiclass', num_classes=10).to(device=device)

AttributeError: module 'torch.nn' has no attribute 'optim'

In [ ]:


# Assuming train_loader is defined
epochs = 5
for epoch in range(epochs):
    loop = tqdm(enumerate(train_loader), total=len(train_loader))
    for idx, (features, targets) in loop:
        features = features.to(device)  # Move data to the device
        targets = targets.to(device)  # Move targets to the device

        # Forward pass
        scores = model(features)
        loss = criterion(scores, targets)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate accuracy (assuming acc is your accuracy function)
        acc_value = acc(scores, targets)

        # Update progress bar
        loop.set_description(f"Epoch [{epoch}/{epochs}]")
        loop.set_postfix(loss=loss.item(), accuracy=acc_value.item())